<a href="https://colab.research.google.com/github/danielhcg/AntiSmishGPT/blob/main/llama_smishTank_Testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [31]:
pip install llamaapi

In [32]:
pip install tqdm


In [41]:
import pandas as pd
import json
from llamaapi import LlamaAPI
from google.colab import userdata
from tqdm import tqdm
import re  # Ensure 're' module is imported


# Initialize the SDK
llama = LlamaAPI(userdata.get('LLAMA-API-KEY'))


In [74]:
def test_api(message):
    response = llama.run({"messages": [{"role": "user", "content": message}]}).json()
    content = response['choices'][0]['message']['content']
    print(f"API response content: {content}")

# Test message
message = "Costco: Daniel, the code 42003 printed on your receipt from 10 came in 2nd in our Airpods draw: f2gpy.info/RzNKEws Zve"
test_message = f"Is this message real or fake. Just say real, fake, or undetermined: {message}"
test_api(test_message)

API response content: Fake


In [76]:
def check_sms_legitimacy(message):
    request_message = f"Is this message real or fake. Just say real, fake, or undetermined: {message}"
    response = llama.run({"messages": [{"role": "user", "content": request_message}]}).json()
    content = response['choices'][0]['message']['content']
    return content

# Read messages from an Excel file
input_file = 'sms_messages.xlsx'
df = pd.read_excel(input_file, header=None)
df.columns = ['message']

# Process each message and get legitimacy determination
determinations = []
for i, message in enumerate(df['message'], start=1):
    determination = check_sms_legitimacy(message)
    determinations.append(determination)
    print(f"Message {i}: {determination}")

# Add the determinations to the DataFrame
df['determination'] = determinations

# Save the results to a new Excel file
output_file = 'sms_legitimacy_results.xlsx'
df.to_excel(output_file, index=False)

print("Legitimacy check completed. Results saved to:", output_file)

Message 1: Fake
Message 2: Fake
Message 3: Fake
Message 4: Fake
Message 5: Real
Message 6: Fake
Message 7: Fake
Message 8: Fake
Message 9: Fake
Message 10: Real
Message 11: Fake
Message 12: Fake
Message 13: Fake
Message 14: Fake
Message 15: Fake
Message 16: Real
Message 17: Fake
Message 18: Fake
Message 19: Fake
Message 20: Fake
Message 21: Fake
Message 22: Fake
Message 23: Fake
Message 24: Fake
Message 25: Fake
Message 26: Fake
Message 27: Fake
Message 28: Fake
Message 29: Fake
Message 30: Fake
Message 31: Fake
Message 32: Fake
Message 33: Fake
Message 34: Fake
Message 35: Fake
Message 36: Fake
Message 37: Fake
Message 38: Fake
Message 39: Fake
Message 40: Fake
Legitimacy check completed. Results saved to: sms_legitimacy_results.xlsx


In [ ]:
def check_sms_legitimacy(message):
    request = {
        "messages": [{"role": "user", "content": f"Is the following SMS message real or fake? {message}"}],
        "functions": [
            {
                "name": "check_sms_legitimacy",
                "description": "Determine if an SMS message is legitimate or fake",
                "parameters": {"type": "object", "properties": {"message": {"type": "string"}}},
                "required": ["message"],
            }
        ],
        "stream": False,
        "function_call": "check_sms_legitimacy",
    }

    response = llama.run(request).json()
    print(f"Raw API response for message '{message}': {json.dumps(response, indent=2)}")

    try:
        arguments = json.loads(response['choices'][0]['message']['function_call']['arguments'])
        result = arguments.get('result')
        if result.lower() in ['real', 'fake']:
            return result.lower()
        else:
            raise ValueError("Unexpected result value")
    except (KeyError, json.JSONDecodeError, ValueError) as e:
        print(f"Error parsing arguments: {e}")
        return 'error'

# Example SMS message
message = "Costco: Daniel, the code 42003 printed on your receipt from 10 came in 2nd in our Airpods draw: f2gpy.info/RzNKEws Zve"
determination = check_sms_legitimacy(message)
print(f"Message: {message}")
print(f"Determination: {determination}")